In [173]:
import pandas as pd
import numpy as np
import gc
path='/home/student303/zz/almama/'
train = pd.read_hdf(path+ 'somefeature/base_feature_train_0.h5')
test = pd.read_hdf(path+ 'somefeature/base_feature_test_0.h5')
print('Read Data Finish')

Read Data Finish


In [174]:
combine = [train, test]
co_df = pd.concat(combine, axis=0, ignore_index=True)

In [119]:
for j in list(train.columns):
    print(co_df[~co_df[j].duplicated()].shape, end='*****')
    print(j)

(496455, 32)*****instance_id
(10236, 32)*****item_id
(14, 32)*****item_category_list
(11181, 32)*****item_property_list
(2075, 32)*****item_brand_id
(128, 32)*****item_city_id
(14, 32)*****item_price_level
(18, 32)*****item_sales_level
(18, 32)*****item_collected_level
(22, 32)*****item_pv_level
(207641, 32)*****user_id
(4, 32)*****user_gender_id
(9, 32)*****user_age_level
(5, 32)*****user_occupation_id
(12, 32)*****user_star_level
(496482, 32)*****context_id
(297416, 32)*****context_timestamp
(20, 32)*****context_page_id
(82378, 32)*****predict_category_property
(4003, 32)*****shop_id
(25, 32)*****shop_review_num_level
(13021, 32)*****shop_review_positive_rate
(22, 32)*****shop_star_level
(18102, 32)*****shop_score_service
(18151, 32)*****shop_score_delivery
(18204, 32)*****shop_score_description
(3, 32)*****is_trade
(14, 32)*****item_category_hash
(8, 32)*****day
(24, 32)*****hour
(60, 32)*****min
(60, 32)*****second


In [175]:
co_df_columns = list(co_df.columns)
for i in ['instance_id','context_id',
          'context_timestamp',
          'predict_category_property',
          'is_trade'
          ,'day'
         ]:
    co_df_columns.remove(i)

In [176]:
co_df_columns

['context_page_id',
 'hour',
 'item_brand_id',
 'item_category_hash',
 'item_category_list',
 'item_city_id',
 'item_collected_level',
 'item_id',
 'item_price_level',
 'item_property_list',
 'item_pv_level',
 'item_sales_level',
 'min',
 'second',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_score_delivery',
 'shop_score_description',
 'shop_score_service',
 'shop_star_level',
 'user_age_level',
 'user_gender_id',
 'user_id',
 'user_occupation_id',
 'user_star_level']

In [177]:
name = locals()
for starttime in range(18,23):
    windowsize = 3
    windowday = []
    for i in range(windowsize):
        windowday.append(starttime + i)
    tmpdata = pd.DataFrame()  
    for i in windowday:
        tmpdata = pd.concat([tmpdata,co_df[co_df.day == i]], axis=0, ignore_index=True)

    name['merge' + '_' + str(starttime + windowsize)] = co_df[co_df.day == (starttime + windowsize)]
    for i in co_df_columns:
        tmpmerge1 = tmpdata[[i,'instance_id']].groupby([i]).count().reset_index()
        tmpmerge1.rename(columns={'instance_id':'A'}, inplace = True)

        tmpmerge2 = tmpdata[tmpdata.is_trade == 1][[i,'instance_id']].groupby([i]).count().reset_index()
        tmpmerge2.rename(columns={'instance_id':'B'}, inplace = True)

        tmpmerge1 = pd.merge(tmpmerge1, tmpmerge2, on=i, how='left')
        tmpmerge1 = tmpmerge1.fillna(0)

        tmpmerge1[i + '_cvr'] = (tmpmerge1['B']+1)  / (tmpmerge1['A']+50)
        tmpmerge1.drop(['A','B'], axis=1, inplace=True)
        name['merge' +  '_' +str(starttime + windowsize)] = pd.merge(name['merge' +  '_' +str(starttime + windowsize)] , tmpmerge1, on=i, how='left')
        del tmpmerge1
        del tmpmerge2 
        gc.collect()

In [178]:
tmpdf =  pd.DataFrame() 
for i in range(21,26):
    tmpdf = pd.concat([tmpdf,name['merge'+'_'+str(i)]], axis=0, ignore_index=True)

## save data to somefeature/one_column_cvr.h5

In [181]:
save_train = tmpdf[tmpdf.day != 25]
save_test = tmpdf[tmpdf.day == 25]

tmpdf.to_hdf(path + 'somefeature/one_column_cvr.h5',key='a',index=False)
save_train.to_hdf(path + 'somefeature/one_column_train_cvr.h5',key='a',index=False)
save_test.to_hdf(path + 'somefeature/one_column_test_cvr.h5',key='a',index=False)